In [4]:
%pip install seaborn

                                              0.0/293.3 kB ? eta -:--:--
     ------------------------------------- 293.3/293.3 kB 17.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizerFast
from sklearn.metrics import confusion_matrix
import numpy as np

# Load validation data
print("Loading dataset...")
df_val = pd.read_csv('../data/validation.csv')
texts = df_val['tweet_text'].tolist()  # Replace 'text' with the name of the column containing the texts
labels = df_val['cyberbullying_type'].tolist()  # Replace 'label' with the name of the column containing the labels

Loading dataset...


In [13]:
# Load the model and tokenizer
print("Loading model...")
model = tf.keras.models.load_model('../models/bert/tf_saved_model/')
print("Loading tokenizer...")
tokenizer = BertTokenizerFast.from_pretrained('../models/bert/tokenizer/')

Loading model...
Loading tokenizer...


In [17]:
# Tokenize the texts
print("Tokenizing...")
val_encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)

# If 'token_type_ids' are not present in the encodings, add them
if 'token_type_ids' not in val_encodings:
    val_encodings['token_type_ids'] = [[0]*len(i) for i in val_encodings['input_ids']]

Tokenizing...


In [21]:
# Prepare the validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((
    {key: np.array(val_encodings[key]) for key in ['input_ids', 'attention_mask', 'token_type_ids']},
    labels
))

In [22]:
# Use the model to predict the labels
true_labels = []
pred_labels = []

for batch in val_dataset.batch(1):
    inputs, label = batch
    preds = model(inputs)
    preds = tf.argmax(preds['logits'], axis=1)
    true_labels.append(label.numpy()[0])
    pred_labels.append(preds.numpy()[0])

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)

print(conf_matrix)


AttributeError: 'dict' object has no attribute 'logits'